In [1]:
import main

In [2]:
import numpy as np
import time

In [3]:
precursor_1 = main.precursor_FDM(delay_group_num=1)
precursor_1.set_beta([0.0064])
precursor_1.set_lambda([0.08])

In [4]:
material_1 = main.material_FDM(mat_id=0)
material_1.set_XS('XS_scatter',[[0,0.01],[0,0]])
material_1.set_XS('Diffusion_coef',[1.4,0.4])
material_1.set_XS('XS_absorption',[0.01,0.15])
# material_1.set_XS('XS_absorption',[0.01,0.10392197])
material_1.set_XS('XS_nu_fission',[0.007,0.2])
material_1.set_XS('XS_fission_spectrum',[1.0,0.0])

In [5]:
material_2 = main.material_FDM(mat_id=1)
material_2.set_XS('XS_scatter',[[0,0.01],[0,0]])
material_2.set_XS('Diffusion_coef',[1.4,0.4])
material_2.set_XS('XS_absorption',[0.01,0.15])
# material_2.set_XS('XS_absorption',[0.02,0.15])
material_2.set_XS('XS_nu_fission',[0.007,0.2])
material_2.set_XS('XS_fission_spectrum',[1.0,0.0])

In [6]:
material_3 = main.material_FDM(mat_id=2)
material_3.set_XS('XS_scatter',[[0,0.01],[0,0]])
material_3.set_XS('Diffusion_coef',[1.3,0.5])
material_3.set_XS('XS_absorption',[0.008,0.05])
# material_3.set_XS('XS_absorption',[0.018,0.05])
material_3.set_XS('XS_nu_fission',[0.003,0.06])
material_3.set_XS('XS_fission_spectrum',[1.0,0.0])

In [7]:
geo = main.geometry_FDM(x_block=3,y_block=3)
sl = slice(0, 3, None)
geo.set_block_mat(material_3,sl,sl)
geo.set_block_mat(material_1,1,1)
geo.set_block_mat(material_2,1,0)
geo.set_block_mat(material_2,0,1)

geo.set_block_size(x_size=[24,32,24],y_size=[24,32,24])
geo.set_discretized_num(x_dim=[24,32,24],y_dim=[24,32,24])
# geo.set_discretized_num(x_dim=[48,64,48],y_dim=[48,64,48])
geo.check_blocks()

Material layout:
[[2 1 2]
 [1 0 2]
 [2 2 2]]
Block size:
x:[24, 32, 24]
y:[24, 32, 24]
Discretization number:
x:[24, 32, 24]
y:[24, 32, 24]


In [8]:
case = main.solver_FDM(folder_name='TWIGL_Seed_Blanket_2D_SCM_A4_adaptive_01_normal',group_num=2)
case.add_material([material_1,material_2,material_3])
case.set_geometry(geo,transient_mode=True)
case.set_precursor(precursor_1)
case.set_neutron_velocity([1e7,1e5])

In [9]:
# Key time steps
fix_time_steps = np.array([0.0,0.2,0.4,0.5]) 
# fix_time_steps = np.array([0.0,0.2]) 
time_index = 0
time_step = fix_time_steps[0]
# Max and min time steps
max_time_step = 5e-2
min_time_step = 1e-3
# Time pointer
key_time_pointer = 1
#Error tolerance
error_tolerance = 1e-3
# Selection strategy
selection_mode = 'Normal-estimate'
# Begin calculation
while True:
    if time_index==0:
        # To get initial flux distribution
        case.solve_source_iter_correct(max_iter=50,k_tolerance=1e-5,flux_tolerance=1e-4,initial_k=1.0)
        case.solve_source_adjoint_iter_correct(max_iter=50,k_tolerance=1e-5,flux_tolerance=1e-4,initial_k=1.0)
        # case.initial_dynamics(transient_algorithm='Implicit_Euler',vtk_save=False)
        time_start=time.time()
        # case.initial_dynamics(time_steps=time_steps,transient_algorithm='SCM',vtk_save=False)
        case.initial_dynamics(time_steps=fix_time_steps,transient_algorithm='SCM_adaptive',vtk_save=True,max_time_step=max_time_step,min_time_step=min_time_step)
    else:
        # Update material
        if time_step<=0.2:
            m1_a2 = 0.15+(time_step-0.0)*(0.15-0.141285)/(0.0-0.2) 
        elif time_step>0.2 and time_step<=0.4:
            m1_a2 = 0.15175+(time_step-0.2)*(0.15175-0.1535)/(0.2-0.4)
        else:
            m1_a2 = 0.15
        material_1.set_XS('XS_absorption',[0.01,m1_a2])
        case.update_material(0,material_1,mat_type='real')
        # Get adaptive time interval 
#         time_interval = case.evaluate_adaptive_time_step(time_index,pre_time_interval=time_interval,error_tolerance=error_tolerance)
        time_interval = case.evaluate_adaptive_time_step_exact(time_index,pre_time_interval=time_interval,error_tolerance=error_tolerance,mode=selection_mode)
        time_step = time_step_last+time_interval
        # Move the time pointer to the next key time step
        if time_step>=fix_time_steps[key_time_pointer]:
            key_time_pointer+=1
        
        print('Time={} (s)'.format(time_step))
        # Re-update material with new time step
        if time_step<=0.2:
            m1_a2 = 0.15+(time_step-0.0)*(0.15-0.141285)/(0.0-0.2) 
        elif time_step>0.2 and time_step<=0.4:
            m1_a2 = 0.15175+(time_step-0.2)*(0.15175-0.1535)/(0.2-0.4)
        else:
            m1_a2 = 0.15
        material_1.set_XS('XS_absorption',[0.01,m1_a2])
        case.update_material(0,material_1,mat_type='real')  
        # SCM solver
#         case.solve_transient_SCM_Adaptive_No_Pred(time_index,time_interval,max_iter=100,k_tolerance=1e-6,flux_tolerance=1e-4,k_outer_tolerance=1e-6)
        case.solve_transient_SCM_PKE_predictor(time_index,time_interval,max_iter=100,k_tolerance=1e-6,flux_tolerance=1e-4,k_outer_tolerance=1e-6) # SCM with PKE predictor
#         case.solve_transient_SCM_2(time_index,time_interval,max_iter=100,k_tolerance=1e-6,flux_tolerance=1e-4,k_outer_tolerance=1e-6,call_type='Normal')
    # TODO
    time_index += 1
    time_step_last = time_step
    # End or not
    if time_step_last>=fix_time_steps[-1]:
        break
    # Pre-choose the last time step
    if fix_time_steps[key_time_pointer]-time_step_last>=max_time_step:
        time_step += max_time_step 
        time_interval = max_time_step
    else:
        time_step = fix_time_steps[key_time_pointer]
        time_interval = fix_time_steps[key_time_pointer]-time_step_last
        

time_end=time.time()
print('totally cost:',time_end-time_start)

D:\Users\85136\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


####################################
Timestep 0 calculation begins
Iteration 1: Eigenvalue k = 0.8446669513193322
Iteration 2: Eigenvalue k = 0.8899489163397002
Iteration 3: Eigenvalue k = 0.8994594776166441
Iteration 4: Eigenvalue k = 0.9038026479314235
Iteration 5: Eigenvalue k = 0.9064220494043876
Iteration 6: Eigenvalue k = 0.9081900679307713
Iteration 7: Eigenvalue k = 0.9094479494340285
Iteration 8: Eigenvalue k = 0.9103687896329571
Iteration 9: Eigenvalue k = 0.9110545506517275
Iteration 10: Eigenvalue k = 0.9115708863732314
Iteration 11: Eigenvalue k = 0.9119625156101641
Iteration 12: Eigenvalue k = 0.9122610517725338
Iteration 13: Eigenvalue k = 0.9124894190761463
Iteration 14: Eigenvalue k = 0.9126645386803547
Iteration 15: Eigenvalue k = 0.9127990586739625
Iteration 16: Eigenvalue k = 0.9129025182983143
Iteration 17: Eigenvalue k = 0.9129821584390484
Iteration 18: Eigenvalue k = 0.9130435007625038
Iteration 19: Eigenvalue k = 0.9130907698523009
Iteration 20: Eigenvalue k = 0

####################################
Outer iteration 2 begins
Iteration 1: Eigenvalue k = 0.9999779644920412
Iteration 2: Met the criteria and k = 0.9999782670090271
Outer Iteration 2： kD=0.9999782670090271
Amplitude frequency last:6.626051943689957
Amplitude frequency:6.692312463126856
Amplitude frequency new (secant):6.559791424253056
Total Q: 1.0829464878821162
####################################
Outer iteration 3 begins
Iteration 1: Eigenvalue k = 0.9999935405970606
Iteration 2: Met the criteria and k = 0.9999930658444536
Outer Iteration 3： kD=0.9999930658444536
Amplitude frequency last:6.692312463126856
Amplitude frequency:6.559791424253056
Amplitude frequency new (secant):6.497697247904343
Total Q: 1.082537771640361
####################################
Outer iteration 4 begins
Iteration 1: Eigenvalue k = 0.9999998751707931
Iteration 2: Met the criteria and k = 0.9999995374654622
Outer Iteration 4： kD=0.9999995374654622
Amplitude frequency last:6.559791424253056
Amplitude frequen

####################################
Outer iteration 4 begins
Iteration 1: Eigenvalue k = 0.999996046325839
Iteration 2: Eigenvalue k = 0.9999976280214807
Iteration 3: Met the criteria and k = 0.9999983579616548
Outer Iteration 4： kD=0.9999983579616548
Amplitude frequency last:12.16761692996399
Amplitude frequency:12.593998227463475
Amplitude frequency new (secant):12.578280593902992
Total Q: 1.9736419006375179
####################################
Outer iteration 5 begins
Iteration 1: Eigenvalue k = 1.000000481742776
Iteration 2: Met the criteria and k = 1.000000506159081
Outer Iteration 5： kD=1.000000506159081
Amplitude frequency last:12.593998227463475
Amplitude frequency:12.578280593902992
Amplitude frequency new (secant):12.581983988601548
Total Q: 1.9737110067352934
Reached the stopping criterion
Total Q (convergent): 1.9737110067352934
Reactivity: 0.004031352819641949
Predicted reactivity (1st): 0.006426647665498202
Optimized time step: 0.01634694874731586
Choosen time step: 0.01

Iteration 10: Eigenvalue k = 1.0000006370222296
Iteration 11: Eigenvalue k = 0.9999990054861582
Iteration 12: Eigenvalue k = 0.9999977329300815
Iteration 13: Met the criteria and k = 0.9999967425373915
Outer Iteration 2： kD=0.9999967425373915
Amplitude frequency last:20.913079688000046
Amplitude frequency:21.122210484880046
Amplitude frequency new (secant):20.703948891120046
Total Q: 4.092772892504799
####################################
Outer iteration 3 begins
Iteration 1: Eigenvalue k = 1.000033505854553
Iteration 2: Eigenvalue k = 1.0000316918689613
Iteration 3: Met the criteria and k = 1.0000307475079806
Outer Iteration 3： kD=1.0000307475079806
Amplitude frequency last:21.122210484880046
Amplitude frequency:20.703948891120046
Amplitude frequency new (secant):21.08214365115965
Total Q: 4.1034029466674005
####################################
Outer iteration 4 begins
Iteration 1: Eigenvalue k = 0.9999965016843301
Iteration 2: Eigenvalue k = 0.9999978835428621
Iteration 3: Met the cri

...

####################################
Outer iteration 1 begins
Iteration 1: Eigenvalue k = 0.999627572500465
Iteration 2: Eigenvalue k = 0.9996394212771451
Iteration 3: Eigenvalue k = 0.9996461581390103
Iteration 4: Eigenvalue k = 0.9996501879233866
Iteration 5: Eigenvalue k = 0.9996527341230939
Iteration 6: Eigenvalue k = 0.9996544161069814
Iteration 7: Eigenvalue k = 0.9996555655910294
Iteration 8: Met the criteria and k = 0.9996563718000641
Outer Iteration 1： kD=0.9996563718000641
Amplitude frequency last:-6.241504541485138
Amplitude frequency:-2.556648252446416
Amplitude frequency new (secant):-2.58221473497088
Total Q: 0.8423574650160394
####################################
Outer iteration 2 begins
Iteration 1: Eigenvalue k = 0.9999908498013775
Iteration 2: Met the criteria and k = 0.999991456059354
Outer Iteration 2： kD=0.999991456059354
Amplitude frequency last:-2.556648252446416
Amplitude frequency:-2.58221473497088
Amplitude frequency new (secant):-2.531081769921952
Total Q: 0.

Iteration 7: Eigenvalue k = 0.9996816491848757
Iteration 8: Eigenvalue k = 0.9996856136599745
Iteration 9: Eigenvalue k = 0.9996888579521371
Iteration 10: Eigenvalue k = 0.9996914587688646
Iteration 11: Eigenvalue k = 0.9996935166170599
Iteration 12: Eigenvalue k = 0.9996951308638984
Iteration 13: Eigenvalue k = 0.9996963897767146
Iteration 14: Met the criteria and k = 0.9996973676500152
Outer Iteration 1： kD=0.9996973676500152
Amplitude frequency last:-0.9576877425685301
Amplitude frequency:-1.1138122884560737
Amplitude frequency new (secant):-1.1249504113406346
Total Q: 0.7685452054778361
####################################
Outer iteration 2 begins
Iteration 1: Eigenvalue k = 1.0000375749877504
Iteration 2: Met the criteria and k = 1.0000378811861743
Outer Iteration 2： kD=1.0000378811861743
Amplitude frequency last:-1.1138122884560737
Amplitude frequency:-1.1249504113406346
Amplitude frequency new (secant):-1.102674165571513
Total Q: 0.7688526910934258
##############################

Iteration 12: Eigenvalue k = 1.0009764135343266
Iteration 13: Eigenvalue k = 1.0009614899224346
Iteration 14: Eigenvalue k = 1.0009498305254596
Iteration 15: Eigenvalue k = 1.0009407571570723
Iteration 16: Eigenvalue k = 1.0009337158158504
Iteration 17: Eigenvalue k = 1.0009282621901396
Iteration 18: Eigenvalue k = 1.0009240442248464
Iteration 19: Eigenvalue k = 1.0009207852440447
Iteration 20: Eigenvalue k = 1.0009182690420577
Iteration 21: Eigenvalue k = 1.0009163273386898
Iteration 22: Eigenvalue k = 1.0009148295248491
Iteration 23: Eigenvalue k = 1.0009136744332496
Iteration 24: Met the criteria and k = 1.0009127838144307
Outer Iteration 1： kD=1.0009127838144307
Amplitude frequency last:-0.9104262757267081
Amplitude frequency:36.59610327100202
Amplitude frequency new (secant):36.96206430371204
Total Q: 0.8395768198985927
####################################
Outer iteration 2 begins
Iteration 1: Eigenvalue k = 0.9996885477032638
Iteration 2: Eigenvalue k = 0.9996915084299376
Iterati

####################################
Outer iteration 4 begins
Iteration 1: Eigenvalue k = 0.9999943299273537
Iteration 2: Eigenvalue k = 0.9999981600932167
Iteration 3: Eigenvalue k = 1.0000000643284441
Iteration 4: Met the criteria and k = 1.0000009820323847
Outer Iteration 4： kD=1.0000009820323847
Amplitude frequency last:9.080567472952792
Amplitude frequency:10.004743866852783
Amplitude frequency new (secant):10.01542095111049
Total Q: 1.0236204300095928
Reached the stopping criterion
Total Q (convergent): 1.0236204300095928
Reactivity: 0.0
Predicted reactivity (1st): 0.0
Optimized time step: 0.007540774467924685
Choosen time step: 0.007540774467924685
Time=0.425420629242517 (s)
Predicted reactivity: 0.0
PKE omega: 9.976350662241671, Real omega: 10.01542095111049
Predicted amplitude frequency: 4.044162919306787
####################################
Timestep 37 calculation begins
Total Q (last): 1.0236204300095928
####################################
Outer iteration 1 begins
Iteration

####################################
Outer iteration 1 begins
Iteration 1: Eigenvalue k = 1.0000377308715658
Iteration 2: Eigenvalue k = 1.0000357465259524
Iteration 3: Met the criteria and k = 1.0000347984942586
Outer Iteration 1： kD=1.0000347984942586
Amplitude frequency last:0.30067498242667956
Amplitude frequency:-0.1772211914134781
Amplitude frequency new (secant):-0.1789934033276129
Total Q: 1.1370465091952444
####################################
Outer iteration 2 begins
Iteration 1: Eigenvalue k = 1.0000210163789185
Iteration 2: Met the criteria and k = 1.000020802880685
Outer Iteration 2： kD=1.000020802880685
Amplitude frequency last:-0.1772211914134781
Amplitude frequency:-0.1789934033276129
Amplitude frequency new (secant):-0.17544897949934335
Total Q: 1.1371159927271672
####################################
Outer iteration 3 begins
Iteration 1: Met the criteria and k = 1.0000200583500685
Outer Iteration 3： kD=1.0000200583500685
Amplitude frequency last:-0.1789934033276129
Amp